# DC EITC and CTC Reforms Analysis (2026)

This notebook analyzes the impact of two tax reforms in 2026:

## Baseline
- DC CTC is repealed (set to $0)
- DC EITC remains at 85% federal match

## Reforms
1. **EITC Enhancement**: Increase DC EITC from 85% to 100% federal match
2. **New CTC**: $1,000 per child under 18, with income thresholds and phase-outs
3. **Combined**: Both EITC and CTC reforms together

## Metrics
For each reform, we calculate:
- Budgetary impact (net cost)
- Overall poverty impact
- Child poverty impact

In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np

c:\Users\dtsax\envs\pe\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Helper Functions

In [2]:
def calculate_poverty(sim, period=2026, child_only=False):
    """
    Calculate poverty rate and count.
    
    Args:
        sim: Microsimulation object
        period: Year to analyze
        child_only: If True, only count children under 18
    
    Returns:
        poverty_rate: Weighted poverty rate (for accurate population estimates)
        people_in_poverty: Unweighted count (number of people in the sample)
    """
    age = sim.calculate("age", period=period)
    is_in_poverty = sim.calculate("person_in_poverty", period=period)
    person_weight = sim.calculate("person_weight", period=period)
    
    if child_only:
        mask = age < 18
    else:
        mask = np.ones_like(age, dtype=bool)
    
    # Weighted poverty rate (correct for population-level analysis)
    weighted_in_poverty = (is_in_poverty[mask] * person_weight[mask]).sum()
    weighted_total = person_weight[mask].sum()
    poverty_rate = weighted_in_poverty / weighted_total if weighted_total > 0 else 0
    
    # Unweighted count (actual sample count for "people lifted")
    unweighted_in_poverty = is_in_poverty[mask].sum()
    unweighted_total = mask.sum()
    
    return {
        "poverty_rate": poverty_rate,
        "people_in_poverty": unweighted_in_poverty,
        "total_people": unweighted_total
    }

def calculate_budgetary_impact(baseline_sim, reform_sim, variable, period=2026):
    """
    Calculate the budgetary impact (net cost) of a reform.
    """
    baseline_value = baseline_sim.calculate(variable, period=period, map_to="household").sum()
    reform_value = reform_sim.calculate(variable, period=period, map_to="household").sum()
    
    return reform_value - baseline_value

def format_currency(value):
    """Format value as currency in millions."""
    return f"${value/1e6:.2f}M"

def format_percent(value):
    """Format value as percentage."""
    return f"{value*100:.2f}%"

## Define Baseline and Reforms

In [3]:
def create_baseline():
    """Baseline: CTC repealed, EITC at 85%"""
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2026-01-01.2100-12-31": 0
            }
        },
        country_id="us",
    )

def create_eitc_reform():
    """EITC only: Increase to 100% match, keep CTC at $0"""
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.eitc.with_children.match": {
                "2026-01-01.2100-12-31": 1.0
            },
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2026-01-01.2100-12-31": 0
            }
        },
        country_id="us",
    )

def create_ctc_reform():
    """CTC only: New $1,000 CTC, keep EITC at 85%"""
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2026-01-01.2100-12-31": 1000
            },
            "gov.states.dc.tax.income.credits.ctc.child.age_threshold": {
                "2026-01-01.2100-12-31": 18
            },
            "gov.states.dc.tax.income.credits.ctc.child.child_cap": {
                "2026-01-01.2100-12-31": 999
            },
            "gov.states.dc.tax.income.credits.ctc.phase_out.amount": {
                "2026-01-01.2100-12-31": 50
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SINGLE": {
                "2026-01-01.2100-12-31": 55_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.JOINT": {
                "2026-01-01.2100-12-31": 70_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SEPARATE": {
                "2026-01-01.2100-12-31": 35_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.HEAD_OF_HOUSEHOLD": {
                "2026-01-01.2100-12-31": 55_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SURVIVING_SPOUSE": {
                "2026-01-01.2100-12-31": 55_000
            }
        },
        country_id="us",
    )

def create_combined_reform():
    """Combined: Both EITC (100%) and new CTC ($1,000)"""
    return Reform.from_dict(
        {
            # EITC: 100% match
            "gov.states.dc.tax.income.credits.eitc.with_children.match": {
                "2026-01-01.2100-12-31": 1.0
            },
            # CTC: $1,000 per child
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2026-01-01.2100-12-31": 1000
            },
            "gov.states.dc.tax.income.credits.ctc.child.age_threshold": {
                "2026-01-01.2100-12-31": 18
            },
            "gov.states.dc.tax.income.credits.ctc.child.child_cap": {
                "2026-01-01.2100-12-31": 999
            },
            "gov.states.dc.tax.income.credits.ctc.phase_out.amount": {
                "2026-01-01.2100-12-31": 50
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SINGLE": {
                "2026-01-01.2100-12-31": 55_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.JOINT": {
                "2026-01-01.2100-12-31": 70_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SEPARATE": {
                "2026-01-01.2100-12-31": 35_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.HEAD_OF_HOUSEHOLD": {
                "2026-01-01.2100-12-31": 55_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SURVIVING_SPOUSE": {
                "2026-01-01.2100-12-31": 55_000
            }
        },
        country_id="us",
    )

print("Reform functions defined!")

Reform functions defined!


## Load Simulations

In [4]:
print("Loading baseline (CTC repealed)...")
baseline_reform = create_baseline()
baseline = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=baseline_reform)
print("✓ Baseline loaded")

print("\nLoading EITC reform...")
eitc_reform = create_eitc_reform()
eitc_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=eitc_reform)
print("✓ EITC reform loaded")

print("\nLoading CTC reform...")
ctc_reform = create_ctc_reform()
ctc_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=ctc_reform)
print("✓ CTC reform loaded")

print("\nLoading combined reform...")
combined_reform = create_combined_reform()
combined_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=combined_reform)
print("✓ Combined reform loaded")

print("\n" + "="*60)
print("All simulations ready!")
print("="*60)

Loading baseline (CTC repealed)...
✓ Baseline loaded

Loading EITC reform...
✓ EITC reform loaded

Loading CTC reform...
✓ CTC reform loaded

Loading combined reform...
✓ Combined reform loaded

All simulations ready!


## Calculate Impacts

In [5]:
# Baseline metrics
baseline_overall_pov = calculate_poverty(baseline, child_only=False)
baseline_child_pov = calculate_poverty(baseline, child_only=True)

# EITC reform impacts
eitc_cost = calculate_budgetary_impact(baseline, eitc_sim, "dc_eitc")
eitc_ctc_cost = calculate_budgetary_impact(baseline, eitc_sim, "dc_ctc")
eitc_total_cost = eitc_cost + eitc_ctc_cost
eitc_overall_pov = calculate_poverty(eitc_sim, child_only=False)
eitc_child_pov = calculate_poverty(eitc_sim, child_only=True)

# CTC reform impacts
ctc_eitc_cost = calculate_budgetary_impact(baseline, ctc_sim, "dc_eitc")
ctc_cost = calculate_budgetary_impact(baseline, ctc_sim, "dc_ctc")
ctc_total_cost = ctc_eitc_cost + ctc_cost
ctc_overall_pov = calculate_poverty(ctc_sim, child_only=False)
ctc_child_pov = calculate_poverty(ctc_sim, child_only=True)

# Combined reform impacts
combined_eitc_cost = calculate_budgetary_impact(baseline, combined_sim, "dc_eitc")
combined_ctc_cost = calculate_budgetary_impact(baseline, combined_sim, "dc_ctc")
combined_total_cost = combined_eitc_cost + combined_ctc_cost
combined_overall_pov = calculate_poverty(combined_sim, child_only=False)
combined_child_pov = calculate_poverty(combined_sim, child_only=True)

print("✓ All impacts calculated")

✓ All impacts calculated


## Results Summary

In [6]:
# Calculate relative changes
def calc_relative_change(baseline_rate, reform_rate):
    if baseline_rate == 0:
        return 0
    return ((baseline_rate - reform_rate) / baseline_rate) * 100

eitc_overall_rel = calc_relative_change(baseline_overall_pov['poverty_rate'], eitc_overall_pov['poverty_rate'])
eitc_child_rel = calc_relative_change(baseline_child_pov['poverty_rate'], eitc_child_pov['poverty_rate'])
ctc_overall_rel = calc_relative_change(baseline_overall_pov['poverty_rate'], ctc_overall_pov['poverty_rate'])
ctc_child_rel = calc_relative_change(baseline_child_pov['poverty_rate'], ctc_child_pov['poverty_rate'])
combined_overall_rel = calc_relative_change(baseline_overall_pov['poverty_rate'], combined_overall_pov['poverty_rate'])
combined_child_rel = calc_relative_change(baseline_child_pov['poverty_rate'], combined_child_pov['poverty_rate'])

# Create comprehensive results table
results = [
    {
        "Scenario": "Baseline",
        "Net Cost": "$0.00M",
        "Overall Poverty Rate": format_percent(baseline_overall_pov['poverty_rate']),
        "Overall Poverty Change": "0.0%",
        "Child Poverty Rate": format_percent(baseline_child_pov['poverty_rate']),
        "Child Poverty Change": "0.0%",
        "People Lifted from Poverty": "0",
        "Children Lifted from Poverty": "0"
    },
    {
        "Scenario": "EITC Only",
        "Net Cost": format_currency(eitc_total_cost),
        "Overall Poverty Rate": format_percent(eitc_overall_pov['poverty_rate']),
        "Overall Poverty Change": f"{eitc_overall_rel:.1f}%",
        "Child Poverty Rate": format_percent(eitc_child_pov['poverty_rate']),
        "Child Poverty Change": f"{eitc_child_rel:.1f}%",
        "People Lifted from Poverty": f"{int(baseline_overall_pov['people_in_poverty'] - eitc_overall_pov['people_in_poverty']):,}",
        "Children Lifted from Poverty": f"{int(baseline_child_pov['people_in_poverty'] - eitc_child_pov['people_in_poverty']):,}"
    },
    {
        "Scenario": "CTC Only",
        "Net Cost": format_currency(ctc_total_cost),
        "Overall Poverty Rate": format_percent(ctc_overall_pov['poverty_rate']),
        "Overall Poverty Change": f"{ctc_overall_rel:.1f}%",
        "Child Poverty Rate": format_percent(ctc_child_pov['poverty_rate']),
        "Child Poverty Change": f"{ctc_child_rel:.1f}%",
        "People Lifted from Poverty": f"{int(baseline_overall_pov['people_in_poverty'] - ctc_overall_pov['people_in_poverty']):,}",
        "Children Lifted from Poverty": f"{int(baseline_child_pov['people_in_poverty'] - ctc_child_pov['people_in_poverty']):,}"
    },
    {
        "Scenario": "Combined (EITC + CTC)",
        "Net Cost": format_currency(combined_total_cost),
        "Overall Poverty Rate": format_percent(combined_overall_pov['poverty_rate']),
        "Overall Poverty Change": f"{combined_overall_rel:.1f}%",
        "Child Poverty Rate": format_percent(combined_child_pov['poverty_rate']),
        "Child Poverty Change": f"{combined_child_rel:.1f}%",
        "People Lifted from Poverty": f"{int(baseline_overall_pov['people_in_poverty'] - combined_overall_pov['people_in_poverty']):,}",
        "Children Lifted from Poverty": f"{int(baseline_child_pov['people_in_poverty'] - combined_child_pov['people_in_poverty']):,}"
    }
]

df_results = pd.DataFrame(results)

print("\n" + "="*160)
print("DC EITC AND CTC REFORMS - 2026 IMPACTS")
print("Baseline: CTC Repealed, EITC at 85% Federal Match")
print("="*160)
print(df_results.to_string(index=False))

# Export to CSV
df_results.to_csv("dc_eitc_ctc_2026_impacts.csv", index=False)
print("\n✓ Exported to: dc_eitc_ctc_2026_impacts.csv")


DC EITC AND CTC REFORMS - 2026 IMPACTS
Baseline: CTC Repealed, EITC at 85% Federal Match
             Scenario Net Cost Overall Poverty Rate Overall Poverty Change Child Poverty Rate Child Poverty Change People Lifted from Poverty Children Lifted from Poverty
             Baseline   $0.00M               23.93%                   0.0%             28.71%                 0.0%                          0                            0
            EITC Only   $6.56M               23.93%                   0.0%             28.71%                 0.0%                          0                            0
             CTC Only  $85.78M               23.57%                   1.5%             27.70%                 3.5%                      2,175                        1,218
Combined (EITC + CTC)  $92.34M               23.56%                   1.5%             27.69%                 3.5%                      2,419                        1,279

✓ Exported to: dc_eitc_ctc_2026_impacts.csv


## Detailed Breakdown

In [7]:
print("\n" + "="*80)
print("DETAILED BREAKDOWN BY PROVISION")
print("="*80)

print("\n" + "-"*80)
print("1. EITC ENHANCEMENT (100% federal match)")
print("-"*80)
print(f"Budgetary Impact:")
print(f"  EITC net cost:              {format_currency(eitc_cost)}")
print(f"  CTC net cost:               {format_currency(eitc_ctc_cost)}")
print(f"  Total net cost:             {format_currency(eitc_total_cost)}")
print(f"\nOverall Poverty Impact:")
print(f"  Baseline poverty rate:      {format_percent(baseline_overall_pov['poverty_rate'])}")
print(f"  Reform poverty rate:        {format_percent(eitc_overall_pov['poverty_rate'])}")
print(f"  Relative change:            {eitc_overall_rel:.1f}%")
print(f"  People lifted from poverty: {int(baseline_overall_pov['people_in_poverty'] - eitc_overall_pov['people_in_poverty']):,}")
print(f"\nChild Poverty Impact:")
print(f"  Baseline poverty rate:      {format_percent(baseline_child_pov['poverty_rate'])}")
print(f"  Reform poverty rate:        {format_percent(eitc_child_pov['poverty_rate'])}")
print(f"  Relative change:            {eitc_child_rel:.1f}%")
print(f"  Children lifted from poverty: {int(baseline_child_pov['people_in_poverty'] - eitc_child_pov['people_in_poverty']):,}")

print("\n" + "-"*80)
print("2. NEW CHILD TAX CREDIT ($1,000 per child under 18)")
print("-"*80)
print(f"Budgetary Impact:")
print(f"  EITC net cost:              {format_currency(ctc_eitc_cost)}")
print(f"  CTC net cost:               {format_currency(ctc_cost)}")
print(f"  Total net cost:             {format_currency(ctc_total_cost)}")
print(f"\nOverall Poverty Impact:")
print(f"  Baseline poverty rate:      {format_percent(baseline_overall_pov['poverty_rate'])}")
print(f"  Reform poverty rate:        {format_percent(ctc_overall_pov['poverty_rate'])}")
print(f"  Relative change:            {ctc_overall_rel:.1f}%")
print(f"  People lifted from poverty: {int(baseline_overall_pov['people_in_poverty'] - ctc_overall_pov['people_in_poverty']):,}")
print(f"\nChild Poverty Impact:")
print(f"  Baseline poverty rate:      {format_percent(baseline_child_pov['poverty_rate'])}")
print(f"  Reform poverty rate:        {format_percent(ctc_child_pov['poverty_rate'])}")
print(f"  Relative change:            {ctc_child_rel:.1f}%")
print(f"  Children lifted from poverty: {int(baseline_child_pov['people_in_poverty'] - ctc_child_pov['people_in_poverty']):,}")

print("\n" + "-"*80)
print("3. COMBINED REFORMS (EITC + CTC)")
print("-"*80)
print(f"Budgetary Impact:")
print(f"  EITC net cost:              {format_currency(combined_eitc_cost)}")
print(f"  CTC net cost:               {format_currency(combined_ctc_cost)}")
print(f"  Total net cost:             {format_currency(combined_total_cost)}")
print(f"\nOverall Poverty Impact:")
print(f"  Baseline poverty rate:      {format_percent(baseline_overall_pov['poverty_rate'])}")
print(f"  Reform poverty rate:        {format_percent(combined_overall_pov['poverty_rate'])}")
print(f"  Relative change:            {combined_overall_rel:.1f}%")
print(f"  People lifted from poverty: {int(baseline_overall_pov['people_in_poverty'] - combined_overall_pov['people_in_poverty']):,}")
print(f"\nChild Poverty Impact:")
print(f"  Baseline poverty rate:      {format_percent(baseline_child_pov['poverty_rate'])}")
print(f"  Reform poverty rate:        {format_percent(combined_child_pov['poverty_rate'])}")
print(f"  Relative change:            {combined_child_rel:.1f}%")
print(f"  Children lifted from poverty: {int(baseline_child_pov['people_in_poverty'] - combined_child_pov['people_in_poverty']):,}")


DETAILED BREAKDOWN BY PROVISION

--------------------------------------------------------------------------------
1. EITC ENHANCEMENT (100% federal match)
--------------------------------------------------------------------------------
Budgetary Impact:
  EITC net cost:              $6.56M
  CTC net cost:               $0.00M
  Total net cost:             $6.56M

Overall Poverty Impact:
  Baseline poverty rate:      23.93%
  Reform poverty rate:        23.93%
  Relative change:            0.0%
  People lifted from poverty: 0

Child Poverty Impact:
  Baseline poverty rate:      28.71%
  Reform poverty rate:        28.71%
  Relative change:            0.0%
  Children lifted from poverty: 0

--------------------------------------------------------------------------------
2. NEW CHILD TAX CREDIT ($1,000 per child under 18)
--------------------------------------------------------------------------------
Budgetary Impact:
  EITC net cost:              $0.00M
  CTC net cost:               $8